In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
df = pd.read_csv("data/data.csv")
df.drop(columns=["id", "Unnamed: 32"], errors="ignore", inplace=True)
df["diagnosis"] = df["diagnosis"].map({"M": 1, "B": 0})

df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]

X.shape, y.shape

((569, 30), (569,))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [8]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel="rbf"),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "ANN (MLP)": MLPClassifier(
        hidden_layer_sizes=(16, 8),
        activation="relu",
        solver="adam",
        max_iter=300,
        random_state=42
    )
}


In [9]:
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc

results

c:\Users\husey\OneDrive\Masaüstü\YSA Proje\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


{'Logistic Regression': 0.9649122807017544,
 'KNN': 0.956140350877193,
 'SVM': 0.9736842105263158,
 'Decision Tree': 0.9298245614035088,
 'ANN (MLP)': 0.9824561403508771}

In [10]:
comparison_df = pd.DataFrame.from_dict(
    results,
    orient="index",
    columns=["Accuracy"]
)

comparison_df

,Accuracy
Logistic Regression,0.964912
KNN,0.956140
SVM,0.973684
Decision Tree,0.929825
ANN (MLP),0.982456


In [13]:
plt.figure(figsize=(8, 4))

bars = plt.bar(
    comparison_df.index,
    comparison_df["Accuracy"]
)

plt.title("Model Karşılaştırması (Accuracy)")
plt.ylabel("Accuracy")
plt.ylim(0.90, 1.0)
plt.grid(axis="y")

# 🔢 BAR ÜSTÜNE DEĞER YAZDIRMA
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height + 0.002,
        f"{height:.3f}",
        ha="center",
        va="bottom",
        fontsize=10
    )

plt.tight_layout()
plt.savefig("outputs/model_comparison_all_models.png")
plt.close()


In [14]:
best_model = models["ANN (MLP)"]
ann_pred = best_model.predict(X_test)
ann_cm = confusion_matrix(y_test, ann_pred)

ann_cm

array([[72,  0],
       [ 2, 40]])

In [16]:
plt.figure(figsize=(5, 4))
plt.imshow(ann_cm, cmap="Blues")
plt.title("ANN Confusion Matrix")
plt.colorbar()

classes = ["Benign", "Malignant"]
plt.xticks([0, 1], classes)
plt.yticks([0, 1], classes)

# 🔢 HÜCRE İÇİ DEĞERLER
for i in range(2):
    for j in range(2):
        plt.text(
            j,
            i,
            ann_cm[i, j],
            ha="center",
            va="center",
            color="black",
            fontsize=12
        )

plt.xlabel("Tahmin")
plt.ylabel("Gerçek")

plt.tight_layout()
plt.savefig("outputs/ann_confusion_matrix.png")
plt.close()


In [17]:
plt.figure(figsize=(6, 4))
plt.plot(best_model.loss_curve_)
plt.title("ANN Eğitim Kayıp (Loss)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)

plt.tight_layout()
plt.savefig("outputs/figure_loss.png")
plt.close()

In [19]:
from sklearn.model_selection import learning_curve
import numpy as np

train_sizes, train_scores, val_scores = learning_curve(
    best_model,
    X,
    y,
    cv=5,
    scoring="accuracy",
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1
)

train_mean = train_scores.mean(axis=1)
val_mean = val_scores.mean(axis=1)

In [20]:
plt.figure(figsize=(7, 5))

plt.plot(train_sizes, train_mean, marker="o", label="Train Accuracy")
plt.plot(train_sizes, val_mean, marker="s", label="Validation Accuracy")

plt.title("ANN Learning Curve (Train vs Validation Accuracy)")
plt.xlabel("Eğitim Örnek Sayısı")
plt.ylabel("Accuracy")
plt.ylim(0.9, 1.0)
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig("outputs/figure_accuracy_learning_curve.png")
plt.close()

In [21]:
train_pred = best_model.predict(X_train)
test_pred = best_model.predict(X_test)

train_acc = accuracy_score(y_train, train_pred)
test_acc = accuracy_score(y_test, test_pred)

train_acc, test_acc


(0.9934065934065934, 0.9824561403508771)

In [22]:
import pandas as pd

tt_df = pd.DataFrame({
    "Dataset": ["Train", "Test"],
    "Accuracy": [train_acc, test_acc]
})

plt.figure(figsize=(5, 4))
bars = plt.bar(tt_df["Dataset"], tt_df["Accuracy"])
plt.title("Train vs Test Doğruluk Karşılaştırması (ANN)")
plt.ylabel("Accuracy")
plt.ylim(0.9, 1.0)
plt.grid(axis="y")

for bar in bars:
    h = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        h + 0.002,
        f"{h:.3f}",
        ha="center",
        va="bottom"
    )

plt.tight_layout()
plt.savefig("outputs/figure_train_test_accuracy.png")
plt.close()


In [23]:
from sklearn.metrics import classification_report

print(classification_report(y_test, best_model.predict(X_test)))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99        72
           1       1.00      0.95      0.98        42

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



In [ ]:
import joblib
import os

os.makedirs("outputs", exist_ok=True)

joblib.dump(
    {
        "model": best_model,
        "scaler": scaler
    },
    "outputs/ann_model.pkl"
)

print("✅ Model + Scaler birlikte kaydedildi")


✅ Model outputs/ann_model.pkl olarak kaydedildi


In [25]:
# Test verisini arayüz için kaydet
X_test_df = pd.DataFrame(X_test, columns=X.columns)
y_test_df = pd.Series(y_test.values, name="diagnosis")

X_test_df.to_csv("outputs/X_test.csv", index=False)
y_test_df.to_csv("outputs/y_test.csv", index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Veri setinin yüklenmesi
df = pd.read_csv("data/data.csv")

# Sınıf dağılımının hesaplanması
class_counts = df["diagnosis"].value_counts()

# Etiketler
labels = ["Benign", "Malignant"]
values = [
    class_counts.get("B", 0),
    class_counts.get("M", 0)
]

# Grafik oluşturma
plt.figure(figsize=(6, 4))
plt.bar(labels, values)
plt.xlabel("Sınıf")
plt.ylabel("Örnek Sayısı")
plt.title("Wisconsin Breast Cancer Diagnostic Veri Seti Sınıf Dağılımı")

# Grafik kaydetme
plt.tight_layout()
plt.savefig("outputs/figure_class_distribution.png", dpi=300)
plt.show()


KeyError: 'target'